In [11]:
import lucem_illud
import pandas as pd
import re, string
from tqdm import tqdm
from bs4 import BeautifulSoup


In [2]:
all_df = pd.read_pickle("D:\\hw\\adopt-proj\\all_df.pkl")

In [12]:

def decontracted(phrase):
    """
    Expand the contracted phrase into normal words
    """
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase) # prime 
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    
    return phrase


def clean_text(df):
    """
    Clean the review texts
    """
    cleaned_post_text = []

    for post_text in tqdm(df['post_text']):
        
        # expand the contracted words
        post_text = decontracted(post_text)
        
        #remove html tags
        post_text = BeautifulSoup(post_text, 'lxml').get_text().strip() # re.sub(r'<.*?>', '', text)
        
        #remove non-alphabetic characters
        post_text = re.sub("[^a-zA-Z]"," ", post_text)
    
        #remove url 
        post_text = re.sub(r'https?://\S+|www\.\S+', '', post_text)
        
        #Removing punctutation, string.punctuation in python consists of !"#$%&\'()*+,-./:;<=>?@[\\]^_{|}~`
        post_text = post_text.translate(str.maketrans('', '', string.punctuation))
        # ''.join([char for char in movie_text_data if char not in string.punctuation])
        
        # remove emails
        post_text = re.sub(r"(^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$)", '', post_text)
    
        cleaned_post_text.append(post_text)

    return cleaned_post_text

In [13]:
all_df['cleaned_post_text'] = clean_text(all_df)

  0%|          | 0/301967 [00:00<?, ?it/s]C:\Users\TheBe\AppData\Local\Temp\ipykernel_8860\3069326551.py:34: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  post_text = BeautifulSoup(post_text, 'lxml').get_text().strip() # re.sub(r'<.*?>', '', text)
  1%|          | 2440/301967 [00:01<02:22, 2095.92it/s]C:\Users\TheBe\AppData\Local\Temp\ipykernel_8860\3069326551.py:34: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  post_text = BeautifulSoup(post_text, 'lxml').get_text().strip() # re.sub(r'<.*?>', '', text)
100%|██████████| 301967/301967 [02:48<00:00, 1789.73it/s]


In [14]:
truth_df = all_df[all_df.loc[:,"user_flair"].astype(str) != "nan"]
truth_df["labels"] = truth_df.is_adoptee.astype(int)

C:\Users\TheBe\AppData\Local\Temp\ipykernel_8860\3595581182.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  truth_df["labels"] = truth_df.is_adoptee.astype(int)


In [15]:
tqdm.pandas()

truth_df['tokenized_sents'] = truth_df['post_text'].progress_apply(lambda x: [lucem_illud.word_tokenize(s) for s in lucem_illud.sent_tokenize(x)])
truth_df['normalized_sents'] = truth_df['tokenized_sents'].apply(lambda x: [lucem_illud.normalizeTokens(s, lemma=False) for s in x])

  0%|          | 0/55487 [00:00<?, ?it/s]

c:\Users\TheBe\anaconda3\Lib\site-packages\spacy\pipeline\lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)
100%|██████████| 55487/55487 [1:02:47<00:00, 14.73it/s] 
C:\Users\TheBe\AppData\Local\Temp\ipykernel_8860\1003399339.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  truth_df['tokenized_sents'] = truth_df['post_text'].progress_apply(lambda x: [lucem_illud.word_tokenize(s) for s in lucem_illud.sent_tokenize(x)])
C:\Users\TheBe\AppData\Local\Temp\ipykernel_8860\1003399339.py:4: SettingWithCopyWarning: 
A value is trying 

In [16]:
truth_df.to_pickle("D:\\hw\\adopt-proj\\truth_df_vect_sents.pkl")